In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from langchain_groq import ChatGroq
from pinecone import Pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import requests
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [23]:
pc = Pinecone(api_key=os.getenv("PINECONEDB_API_KEY"))
pc.create_index(
    name="airag",
    dimension=512,
    metric="cosine",
    spec = ServerlessSpec(cloud="aws", region="us-east-1")
)

In [4]:
model = SentenceTransformer('all-MiniLM-l6-v2')

d:\NextJS AI Projects\ai-professor\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
import google.generativeai as genai
# retrieve the API key from the environment
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
# configure the Gemini API using the retrieved key
genai.configure(api_key=GOOGLE_API_KEY)

import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Johnson explains complex concepts clearly. Her labs are challenging but enlightening.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Brilliant instructor! Prof. Chen's passion for coding is contagious. Best CS class I've taken."},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Decent lectures, but exams are much harder than the material covered in class.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Lee is patient and willing to help. His office hours are incredibly useful.'},
 {'professor': 'Dr. Rachel Martinez',
  'subject': 'Psychology',
  'stars': 5,
  'review': 'Engaging lectures and fascinating experiments. Dr. Martinez is a true expert in her field.'},
 {'professor': 'Prof. Robert Taylor',
  'subject': 'History',
  'stars': 2,
  'review': 'Monoto

In [25]:
# Fetch the API key from the environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Check if the key was loaded correctly
if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found. Please check your .env file.")

# Define the API endpoint URL
embedding_api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}"

# Initialize an empty list to hold the processed data
processed_data = []

# Create content for each review
for review in data["reviews"]:
    response = requests.post(
        embedding_api_url,
        json={
            "contents": [
                {
                    "parts": [
                        {"text": review['review']}  # Pass the review text here
                    ]
                }
            ]
        }
    )

    if response.status_code == 200:
        # Extract the generated content from the response
        candidates = response.json().get('candidates', [])
        if candidates:
            content_parts = candidates[0].get('content', {}).get('parts', [])
            if content_parts:
                generated_content = content_parts[0].get('text', '')
                processed_data.append(
                    {
                        "values": generated_content,  # The generated content
                        "id": review["professor"],
                        "metadata": {
                            "review": review["review"],
                            "subject": review["subject"],
                            "stars": review["stars"],
                        }
                    }
                )
    else:
        print(f"Error {response.status_code}: {response.text}")

# Now you have the processed data with the generated content for each review.

Error 429: {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}

Error 429: {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}

Error 429: {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}



In [26]:
processed_data[0]

{'values': 'This is a great start to a description of Dr. Johnson! Here are some ways you could expand on it:\n\n**Focusing on her teaching style:**\n\n* **"Dr. Johnson has a knack for explaining complex concepts clearly, making even the most challenging topics accessible to all students."** This emphasizes her ability to make difficult subjects understandable.\n* **"Her lectures are engaging and informative, with a focus on real-world applications that bring the material to life."** This highlights her interactive and practical approach to teaching.\n* **"Dr. Johnson\'s enthusiasm for her subject is contagious, inspiring students to delve deeper into the material."** This focuses on her passion and how it motivates her students.\n\n**Expanding on her labs:**\n\n* **"Her labs are designed to push students to think critically and problem-solve, providing valuable hands-on experience that complements the theoretical knowledge gained in lectures."** This emphasizes the practical learning 

In [27]:
def convert_to_embedding(text):
    # Replace this with your actual embedding logic.
    # For example, if you're generating embeddings from a model,
    # use the model's output here.
    return [0.1] * 512  # Example of a non-zero embedding vector

# Process the data and prepare it for upserting to Pinecone
valid_vectors = []
for item in processed_data:
    embedding = convert_to_embedding(item["values"])  # Convert the content to embedding
    
    # Ensure the embedding is not a zero vector
    if any(embedding):  # This checks if there's at least one non-zero value
        item["values"] = embedding
        valid_vectors.append(item)

# Insert the valid embeddings into the Pinecone index
index = pc.Index("airag")
upsert_response = index.upsert(
    vectors=valid_vectors,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())

Upserted count: 27
{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [114]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-l6-v2')

embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-l6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [115]:
for review in data["reviews"]:
    # Generate the embedding using HuggingFace's model
    # embedding = embeddings.embed_documents([review['review']])
    embd2 = model.encode([review['review']]).tolist()
    # len(embd2)
    
    # Now you can use this embedding as needed, for example, sending it to Groq or storing it
    # Assuming you want to use the embedding with Groq
    processed_data.append({
        "id": review["professor"],  # Use a unique identifier
        "values": embd2,  # Use the embedding directly
        "metadata": {  # Optional: Add any additional metadata
            "review": review["review"],
            "subject": review["subject"],
            "stars": str(review["stars"])
        }
    })

In [116]:
processed_data

[{'id': 'Dr. Emily Johnson',
  'values': [[-0.018087778240442276,
    -0.02884804829955101,
    0.0040274327620863914,
    0.05528445541858673,
    -0.03930819779634476,
    -0.04668696969747543,
    -0.06694492697715759,
    0.08260329812765121,
    -0.027768036350607872,
    -0.0020295665599405766,
    -0.07458841800689697,
    0.02017383463680744,
    -0.062157414853572845,
    0.05200393497943878,
    -0.05340474098920822,
    0.011727545410394669,
    0.0004323586472310126,
    0.04236026108264923,
    -0.03366503119468689,
    -0.05838058888912201,
    0.03564783185720444,
    -0.0031688774470239878,
    0.05701789632439613,
    0.012335607782006264,
    -0.03837534412741661,
    0.09156271070241928,
    -0.011447995901107788,
    0.004259657114744186,
    0.054694321006536484,
    -0.01783619448542595,
    -0.02412545122206211,
    0.08092039823532104,
    -0.030613014474511147,
    0.010021735914051533,
    -0.025544771924614906,
    0.059379205107688904,
    -0.014925643801689

In [117]:
index = pc.Index('airag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

TypeError: Column `values` is expected to be a list of floats

In [60]:
# Print the first item to inspect its structure
print(processed_data[0])

# Check and correct the format of `values` in each item
for item in processed_data:
    if not isinstance(item['values'], list) or not all(isinstance(x, float) for x in item['values']):
        raise TypeError(f"Invalid type in 'values': {item['values']}")

# Correct data format if needed
corrected_data = []
for item in processed_data:
    corrected_data.append({
        'id': item['id'],
        'values': [float(x) for x in item['values']],  # Ensure all elements are floats
        'metadata': item.get('metadata', {})
    })

# Now upsert the corrected data
index = pc.Index('airag')
index.upsert(
    vectors=corrected_data,
    namespace="ns"
)


{'id': 'Dr. Emily Johnson', 'metadata': fields {
  key: "subject"
  value {
    string_value: "Physics"
  }
}
fields {
  key: "stars"
  value {
    number_value: 4
  }
}
fields {
  key: "review"
  value {
    string_value: "Dr. Johnson explains complex concepts clearly. Her labs are challenging but enlightening."
  }
}
, 'values': [-0.018087778240442276, -0.02884804829955101, 0.0040274327620863914, 0.05528445541858673, -0.03930819779634476, -0.04668696969747543, -0.06694492697715759, 0.08260329812765121, -0.027768036350607872, -0.0020295665599405766, -0.07458841800689697, 0.02017383463680744, -0.062157414853572845, 0.05200393497943878, -0.05340474098920822, 0.011727545410394669, 0.0004323586472310126, 0.04236026108264923, -0.03366503119468689, -0.05838058888912201, 0.03564783185720444, -0.0031688774470239878, 0.05701789632439613, 0.012335607782006264, -0.03837534412741661, 0.09156271070241928, -0.011447995901107788, 0.004259657114744186, 0.054694321006536484, -0.01783619448542595, -0.0

TypeError: Invalid type in 'values': [[-0.11551908403635025, 0.043105218559503555, -0.009648283012211323, 0.0019489002879709005, 0.009817099198698997, -0.030988816171884537, 0.10550185292959213, 0.04243411868810654, -0.007672047708183527, 0.009457379579544067, 0.02545466646552086, -0.053972549736499786, -0.006495674606412649, 0.005198792088776827, 0.007380932569503784, 0.012515761889517307, -0.059585705399513245, -0.003692182945087552, -0.0625670924782753, 0.0365435965359211, -0.06417161226272583, 0.07123205810785294, 0.08865132927894592, 0.012514814734458923, -0.011480038985610008, 0.012159345671534538, 0.030178366228938103, 0.047102782875299454, -0.026999372988939285, -0.056381214410066605, 0.0054781571961939335, 0.04746949300169945, 0.07125619798898697, -0.018013758584856987, 0.021254807710647583, -0.03548804670572281, -0.010515827685594559, -0.011873509734869003, -0.01754342019557953, 0.029299259185791016, -0.038744933903217316, 0.019512148573994637, 0.03328830376267433, 0.06176215